# Adding variables, categories, information from survey infos and external sources

In this third notebook new columns are added from the starting ones.  
From date variable date and time are splitted, season and the exact part of the day are extracted.  
Web Scraping is used to get main events of the year. To update the year's events it is enaugh to change the endpoint at the end specify the year (e.g. '2023').  
The IDs year of birth is calculated to match the Generation to which the users belong.  
Generation description is also scraped from Wikipedia.  
From the specifics of the survey, it is possible to assign if the states of mind are positive, negative, if they are related to the future, if are perceived globally as a result. 
It is also possible to identify if they concern the individual and personal life (PANAS and Personal Life), the environment around (External Context), if they are general or future.  

In [1]:
# import libraries
import pandas as pd
import numpy as np

In [2]:
# read csv cleaned in translated notebook
survey = pd.read_csv(r'data_clean/02_survey_eng.csv')
print(survey.shape)
survey.head(3)

(641480, 18)


,User ID,Variable,Question,Topic,Answer Label,Answer Value,Age,CAP,City,Num. of Citizens,Highest Education,Survey filling date and time,Seconds to Answer,Sex,User Social Weight,idRispondente,Pet Friends,Num. of Children (0-14 years)
0,221250719,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A lot,5,20,70032,BA,56929.0,High school diploma,2023-02-14 19:06:44,10.849250,Male,5.458265,221250719.0,No,0
1,89508748,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little yes and a little no,3,21,87060,CS,1281.0,High school diploma,2023-02-27 18:36:02,12.965000,Female,5.458265,89508748.0,No,0
2,98143773,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little,2,21,63822,FM,16062.0,High school diploma,2023-02-19 18:39:59,9.831083,Female,5.458265,98143773.0,Sì,0


In [3]:
survey.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 641480 entries, 0 to 641479
Data columns (total 18 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   User ID                        641480 non-null  int64  
 1   Variable                       641480 non-null  object 
 2   Question                       641480 non-null  object 
 3   Topic                          625240 non-null  object 
 4   Answer Label                   641480 non-null  object 
 5   Answer Value                   641480 non-null  int64  
 6   Age                            641480 non-null  int64  
 7   CAP                            641480 non-null  int64  
 8   City                           604666 non-null  object 
 9   Num. of Citizens               636898 non-null  float64
 10  Highest Education              641480 non-null  object 
 11  Survey filling date and time   641480 non-null  object 
 12  Seconds to Answer             

In [4]:
# take a look to null values in 'Seconds to Answer' column to see if there are some relations with other datas
seconds_null = survey[survey['Seconds to Answer'].isnull() == True]
seconds_null.head(10)

,User ID,Variable,Question,Topic,Answer Label,Answer Value,Age,CAP,City,Num. of Citizens,Highest Education,Survey filling date and time,Seconds to Answer,Sex,User Social Weight,idRispondente,Pet Friends,Num. of Children (0-14 years)
70,2073021648,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little yes and a little no,3,27,20121,MI,1256211.0,Degree or doctorate,2023-02-21 20:22:39,NaN,Male,0.356399,2.073022e+09,Sì,2
411,4104689591,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little yes and a little no,3,35,89133,RC,14167.0,Degree or doctorate,2023-02-16 18:28:04,NaN,Female,0.218331,4.104690e+09,No,0
432,1182387311,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little yes and a little no,3,36,10024,TO,53350.0,High school diploma,2023-02-21 11:57:15,NaN,Male,1.036570,1.182387e+09,No,0
704,3210242785,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little,2,40,9041,CA,7979.0,Degree or doctorate,2023-02-24 18:50:53,NaN,Male,0.429012,3.210243e+09,No,0
794,1827579448,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,Quite a lot,4,41,82019,BN,11558.0,Degree or doctorate,2023-02-16 18:28:50,NaN,Female,0.218331,1.827579e+09,Sì,0
800,4188517977,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little yes and a little no,3,41,90015,PA,13789.0,Degree or doctorate,2023-02-15 18:50:48,NaN,Female,0.218331,4.188518e+09,No,0
813,547506152,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,Not at all,1,42,47043,FC,21716.0,High school diploma,2023-02-22 20:49:10,NaN,Male,1.812258,5.475062e+08,Sì,0
933,1387296751,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,Quite a lot,4,43,70037,BA,25741.0,Degree or doctorate,2023-02-15 19:17:00,NaN,Female,0.218331,1.387297e+09,Sì,0
1072,801535583,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,Quite a lot,4,45,24019,BG,9015.0,Middle school diploma,2023-02-17 16:25:37,NaN,Female,1.170403,8.015356e+08,No,0
1124,78446999,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little,2,45,45100,RO,50289.0,Degree or doctorate,2023-02-17 18:26:21,NaN,Female,0.218331,7.844700e+07,Sì,0


In [5]:
# cast column to date
survey['Survey filling date and time'] = [pd.to_datetime(i) for i in survey['Survey filling date and time']]

In [6]:
# create a directory to save mapping dataframes to update future surveys analysis
import os
os.makedirs('data_update', exist_ok=True)

# Adding Time variables

In [7]:
# import specific library to manage date. Extract the season from the day and month in column 'Survey filling date and time'
from datetime import datetime
season = []

for i in survey['Survey filling date and time']:
    if i.strftime('%m-%d') >= '06-21' and i.strftime('%m-%d') < '09-23':
        s = 'Summer'
        season.append(s)
    elif i.strftime('%m-%d') >= '09-23' and i.strftime('%m-%d') < '12-22':
        a = 'Autumn'
        season.append(a)
    elif i.strftime('%m-%d') >= '12-22' and i.strftime('%m-%d') <= '12-31':
        w = 'Winter'
        season.append(w)
    elif i.strftime('%m-%d') >= '01-01' and i.strftime('%m-%d') < '03-20':
        w = 'Winter'
        season.append(w)
    elif i.strftime('%m-%d') >= '03-20' and i.strftime('%m-%d') < '06-21':
        sp = 'Spring'
        season.append(sp)    
    else:
        0  

survey['Season'] = season 

In [8]:
# extract month from 'Survey filling date and time'
month = []

for i in survey['Survey filling date and time'].dt.month:
    if i == 1:
        m = 'January'
    elif i == 2:
        m = 'February'
    elif i == 3:
        m = 'March'
    elif i == 4:
        m = 'April'
    elif i == 5:
        m = 'May'
    elif i == 6:
        m = 'June'
    elif i == 7:
        m = 'July'
    elif i == 8:
        m = 'August'
    elif i == 9:
        m = 'September'
    elif i == 10:
        m = 'October'
    elif i == 11:
        m = 'November'
    elif i == 12:
        m = 'December'         
    else:
        0 
    month.append(m)

survey['Month'] = month

In [9]:
# time slot is extracted from 'Survey filling date and time'
time_slot = []

for h in survey['Survey filling date and time']:
    if h.strftime('%H') >= '00' and h.strftime('%H') < '06':
        t = 'Night'    
    elif h.strftime('%H') >= '06' and h.strftime('%H') < '12':
        t = 'Morning'
    elif h.strftime('%H') >= '12' and h.strftime('%H') < '18':
        t = 'Noon'
    elif h.strftime('%H') >= '18' and h.strftime('%H') <= '23':
        t = 'Evening'  
    else:
        0
    time_slot.append(t)
        
survey['Time Slot'] = time_slot 

In [10]:
# the year of birth of each user is calculated to assign his cultural generation
import datetime

year_birth = [(datetime.date.today().year - i) for i in survey['Age']]
year_range = pd.Series(np.arange(1901, stop=(datetime.date.today().year)+1, dtype=int))

generation_name = []

for a in year_range:
    if a >= 1901 and a <= 1927:
        g = 'Greatest Generation 1901 - 1927'
    elif a > 1927 and a <= 1945:
        g = 'Silent Generation 1928 - 1945'
    elif a > 1945 and a <= 1964:
        g = 'Baby Boomers 1946 - 1964'
    elif a > 1964 and a <= 1980:
        g = 'Generation X 1965 - 1980'
    elif a > 1980 and a <= 1996:
        g = "Millenials 1981 - 1996"
    elif a > 1996 and a <= 2012:
        g = 'Generation Z 1997 - 2012'
    elif a > 2013:
        g = 'Generation Alpha 2013 - present'
    else:
        0
    generation_name.append(g)

generation_name = pd.Series(generation_name)


In [11]:
# import library or web scraping
import requests
from bs4 import BeautifulSoup

In [12]:
# connect to endpoint
url_gen = 'https://en.wikipedia.org/wiki/Generation'
page_gen = requests.get(url_gen)
print(page_gen.content)

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-feature-typography-survey-disabled vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>Generation - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinn

In [13]:
# specify parser for html pages
soup_gen = BeautifulSoup(page_gen.content, 'html.parser')
txt_gen = soup_gen.find_all('li') # find all 'li' html tags which contain the generation description text  
txt_gen

[<li class="mw-list-item" id="n-mainpage-description"><a accesskey="z" href="/wiki/Main_Page" title="Visit the main page [z]"><span>Main page</span></a></li>,
 <li class="mw-list-item" id="n-contents"><a href="/wiki/Wikipedia:Contents" title="Guides to browsing Wikipedia"><span>Contents</span></a></li>,
 <li class="mw-list-item" id="n-currentevents"><a href="/wiki/Portal:Current_events" title="Articles related to current events"><span>Current events</span></a></li>,
 <li class="mw-list-item" id="n-randompage"><a accesskey="x" href="/wiki/Special:Random" title="Visit a randomly selected article [x]"><span>Random article</span></a></li>,
 <li class="mw-list-item" id="n-aboutsite"><a href="/wiki/Wikipedia:About" title="Learn about Wikipedia and how it works"><span>About Wikipedia</span></a></li>,
 <li class="mw-list-item" id="n-contactpage"><a href="//en.wikipedia.org/wiki/Wikipedia:Contact_us" title="How to contact Wikipedia"><span>Contact us</span></a></li>,
 <li class="mw-list-item" id

In [14]:
# create a list of strings with all texts contained in 'li' tags
txt_gen_list = []

for t in txt_gen:
    txt_gen_list.append(t.text)
txt_gen_list

['Main page',
 'Contents',
 'Current events',
 'Random article',
 'About Wikipedia',
 'Contact us',
 'Donate',
 'Help',
 'Learn to edit',
 'Community portal',
 'Recent changes',
 'Upload file',
 'Create account\n',
 'Log in\n',
 ' Create account',
 ' Log in',
 'Contributions',
 'Talk',
 '\n\n(Top)\n\n',
 '\n\n\n1Etymology\n\n\n\n',
 '\n\n\n2Familial generation\n\n\n\n',
 '\n\n\n3Social generation\n\n\n\nToggle Social generation subsection\n\n\n\n\n\n3.1Generational theory\n\n\n\n\n\n\n\n3.2Generational tension\n\n\n\n\n\n',
 '\n\n\n3.1Generational theory\n\n\n\n',
 '\n\n\n3.2Generational tension\n\n\n\n',
 '\n\n\n4List of social generations\n\n\n\nToggle List of social generations subsection\n\n\n\n\n\n4.1Western world\n\n\n\n\n\n\n\n4.2Other areas\n\n\n\n\n\n\n\n4.3Other terminology\n\n\n\n\n\n',
 '\n\n\n4.1Western world\n\n\n\n',
 '\n\n\n4.2Other areas\n\n\n\n',
 '\n\n\n4.3Other terminology\n\n\n\n',
 '\n\n\n5Criticism\n\n\n\n',
 '\n\n\n6See also\n\n\n\n',
 '\n\n\n7References\n\n\n\n

In [15]:
# create a list with strings characterizing only the relevant texts/strings of 'li' tags
txt_descr_id = ['The Lost Generation', 'The Greatest Generation', 'The Silent Generation', 'Baby boomers (often shortened to Boomers)',
         'Generation X (or Gen X for short)', 'Millennials, also known as Generation Y[47]', 
         'Generation Z (or Gen Z for short and colloquially as "Zoomers")','Generation Alpha (or Gen Alpha for short)']

In [16]:
# create new list with only 'li' tags text of generations descriptions 
gen_description = []

for t in txt_gen_list:
    for d in txt_descr_id:
        if d in t :
            gen_description.append(t)
gen_description

['The Lost Generation, also known as the "Generation of 1914" in Europe,[35] is a term originating from Gertrude Stein to describe those who fought in World War I. The Lost Generation is defined as the cohort born from 1883 to 1900 who came of age during World War I and the Roaring Twenties.[36]',
 'The Greatest Generation, also known in American usage as the "G.I. Generation",[37] includes the veterans who fought in World War II. They were born from 1901 to 1927;[38] older G.I.s (or the Interbellum Generation) came of age during the Roaring Twenties, while younger G.I.s came of age during the Great Depression and World War II. Journalist Tom Brokaw wrote about American members of this cohort in his book The Greatest Generation, which popularized the term.[39]',
 'The Silent Generation, also known as the "Lucky Few", is the cohort who came of age in the post–World War II era. They were born from 1928 to 1945.[40][41] In the U.S., this group includes most of those who may have fought th

In [17]:
print(len(gen_description))
gen_description = gen_description[:8]
gen_description

9


['The Lost Generation, also known as the "Generation of 1914" in Europe,[35] is a term originating from Gertrude Stein to describe those who fought in World War I. The Lost Generation is defined as the cohort born from 1883 to 1900 who came of age during World War I and the Roaring Twenties.[36]',
 'The Greatest Generation, also known in American usage as the "G.I. Generation",[37] includes the veterans who fought in World War II. They were born from 1901 to 1927;[38] older G.I.s (or the Interbellum Generation) came of age during the Roaring Twenties, while younger G.I.s came of age during the Great Depression and World War II. Journalist Tom Brokaw wrote about American members of this cohort in his book The Greatest Generation, which popularized the term.[39]',
 'The Silent Generation, also known as the "Lucky Few", is the cohort who came of age in the post–World War II era. They were born from 1928 to 1945.[40][41] In the U.S., this group includes most of those who may have fought th

In [18]:
# match the description with to the related generation
generation_col = []

for i in generation_name:
            if 'Lost Generation' in i:
                generation_col.append(gen_description[0])
            elif 'Greatest Generation 1901 - 1927' in i:
                 generation_col.append(gen_description[1])
            elif 'Silent Generation 1928 - 1945' in i:
                 generation_col.append(gen_description[2])
            elif 'Baby Boomers 1946 - 1964' in i:
                generation_col.append(gen_description[3])
            elif 'Generation X 1965 - 1980' in i:
                generation_col.append(gen_description[4])
            elif 'Millenials 1981 - 1996' in i:
                generation_col.append(gen_description[5])
            elif 'Generation Z 1997 - 2012' in i:
                generation_col.append(gen_description[6])
            elif 'Generation Alpha 2013 - present' in i:
                generation_col.append(gen_description[7])
     
generation_col

['The Greatest Generation, also known in American usage as the "G.I. Generation",[37] includes the veterans who fought in World War II. They were born from 1901 to 1927;[38] older G.I.s (or the Interbellum Generation) came of age during the Roaring Twenties, while younger G.I.s came of age during the Great Depression and World War II. Journalist Tom Brokaw wrote about American members of this cohort in his book The Greatest Generation, which popularized the term.[39]',
 'The Greatest Generation, also known in American usage as the "G.I. Generation",[37] includes the veterans who fought in World War II. They were born from 1901 to 1927;[38] older G.I.s (or the Interbellum Generation) came of age during the Roaring Twenties, while younger G.I.s came of age during the Great Depression and World War II. Journalist Tom Brokaw wrote about American members of this cohort in his book The Greatest Generation, which popularized the term.[39]',
 'The Greatest Generation, also known in American us

In [19]:
generation_col = pd.Series(generation_col)

In [20]:
# dataframe 'generation_mapping' is created to map the cultural generation of each user in the main dataframe
generation_mapping = pd.concat([year_range, generation_name, generation_col], axis=1)
generation_mapping.rename(columns={ 0:'Year', 1:'Generation', 2: 'Generation Description'}, inplace=True)

In [21]:
generation_mapping.head(100)

,Year,Generation,Generation Description
0,1901,Greatest Generation 1901 - 1927,"The Greatest Generation, also known in America..."
1,1902,Greatest Generation 1901 - 1927,"The Greatest Generation, also known in America..."
2,1903,Greatest Generation 1901 - 1927,"The Greatest Generation, also known in America..."
3,1904,Greatest Generation 1901 - 1927,"The Greatest Generation, also known in America..."
4,1905,Greatest Generation 1901 - 1927,"The Greatest Generation, also known in America..."
...,...,...,...
95,1996,Millenials 1981 - 1996,"Millennials, also known as Generation Y[47] (o..."
96,1997,Generation Z 1997 - 2012,Generation Z (or Gen Z for short and colloquia...
97,1998,Generation Z 1997 - 2012,Generation Z (or Gen Z for short and colloquia...
98,1999,Generation Z 1997 - 2012,Generation Z (or Gen Z for short and colloquia...


In [22]:
# replace/remove special characters in gnerartion descriptions
import re
generation_mapping['Generation Description'].replace('\[(\d+)\]',' ', regex=True, inplace=True)
generation_mapping['Generation Description'] = [i.replace('\n', ' ') for i in generation_mapping['Generation Description']]

In [23]:
generation_mapping['Generation Description'].unique()

array(['The Greatest Generation, also known in American usage as the "G.I. Generation",  includes the veterans who fought in World War II. They were born from 1901 to 1927;  older G.I.s (or the Interbellum Generation) came of age during the Roaring Twenties, while younger G.I.s came of age during the Great Depression and World War II. Journalist Tom Brokaw wrote about American members of this cohort in his book The Greatest Generation, which popularized the term. ',
       'The Silent Generation, also known as the "Lucky Few", is the cohort who came of age in the post–World War II era. They were born from 1928 to 1945.   In the U.S., this group includes most of those who may have fought the Korean War and many of those who may have fought during the Vietnam War.',
       'Baby boomers (often shortened to Boomers), are the people born following World War II from 1946 to 1964. Increased birth rates were observed during the post–World War II baby boom, making them a relatively large demog

In [24]:
# save to csv the dataframe with the cultural generations descriptions. 
generation_mapping.to_csv(r'data_update/generation_mapping.csv', index= False)

In [25]:
# merge main dataframe adding the user's cultural generation by 'generation_mapping' dataframe
survey['birth_year'] = [(datetime.date.today().year - i) for i in survey['Age']]

In [26]:
# split columns in 'Survey filling date' and 'Survey filling hour' from the original 'Survey filling date and time' column
survey['Survey filling date'] = [i.strftime('%d/%m/%Y') for i in survey['Survey filling date and time']]
survey['Survey filling hour'] = [i.strftime('%H:%M:%S') for i in survey['Survey filling date and time']]

In [27]:
# extract weekday from the date
import datetime
from datetime import date, datetime
import calendar
day = [datetime.strptime(i, '%d/%m/%Y') for i in survey['Survey filling date']]
day = [datetime.date(i) for i in day]
weekday = [calendar.day_name[i.weekday()] for i in day]


In [28]:
# add weekday column to survey DF
survey['Weekday'] = weekday

In [29]:
print(survey.shape)
survey.head()

(641480, 25)


,User ID,Variable,Question,Topic,Answer Label,Answer Value,Age,CAP,City,Num. of Citizens,...,idRispondente,Pet Friends,Num. of Children (0-14 years),Season,Month,Time Slot,birth_year,Survey filling date,Survey filling hour,Weekday
0,221250719,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A lot,5,20,70032,BA,56929.0,...,221250719.0,No,0,Winter,February,Evening,2003,14/02/2023,19:06:44,Tuesday
1,89508748,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little yes and a little no,3,21,87060,CS,1281.0,...,89508748.0,No,0,Winter,February,Evening,2002,27/02/2023,18:36:02,Monday
2,98143773,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little,2,21,63822,FM,16062.0,...,98143773.0,Sì,0,Winter,February,Evening,2002,19/02/2023,18:39:59,Sunday
3,641447689,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little,2,21,90024,PA,7614.0,...,641447689.0,Sì,1,Winter,February,Evening,2002,28/02/2023,20:32:30,Tuesday
4,61843325,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little,2,21,95024,CT,50190.0,...,61843325.0,No,0,Winter,February,Evening,2002,16/02/2023,18:25:16,Thursday


In [30]:
# extract the current year
import datetime
current_year = str(datetime.date.today().year)

In [31]:
# assign endpoint and get the content of the web page through a request
url = f'https://en.wikipedia.org/wiki/{current_year}'
page_events = requests.get(url)
print(page_events.content)

b'<!DOCTYPE html>\n<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-zebra-design-disabled vector-feature-custom-font-size-clientpref-0 vector-feature-client-preferences-disabled vector-feature-client-prefs-pinned-disabled vector-feature-typography-survey-disabled vector-toc-available" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8">\n<title>2023 - Wikipedia</title>\n<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-dis

In [32]:
# specify the parser 'html.parser' in order to read the web page and find all the html tags 'li' containing the texts of the events of the year
soup_events = BeautifulSoup(page_events.content, 'html.parser')
events_txt = soup_events.find_all('li')
events_txt

[<li class="mw-list-item" id="n-mainpage-description"><a accesskey="z" href="/wiki/Main_Page" title="Visit the main page [z]"><span>Main page</span></a></li>,
 <li class="mw-list-item" id="n-contents"><a href="/wiki/Wikipedia:Contents" title="Guides to browsing Wikipedia"><span>Contents</span></a></li>,
 <li class="mw-list-item" id="n-currentevents"><a href="/wiki/Portal:Current_events" title="Articles related to current events"><span>Current events</span></a></li>,
 <li class="mw-list-item" id="n-randompage"><a accesskey="x" href="/wiki/Special:Random" title="Visit a randomly selected article [x]"><span>Random article</span></a></li>,
 <li class="mw-list-item" id="n-aboutsite"><a href="/wiki/Wikipedia:About" title="Learn about Wikipedia and how it works"><span>About Wikipedia</span></a></li>,
 <li class="mw-list-item" id="n-contactpage"><a href="//en.wikipedia.org/wiki/Wikipedia:Contact_us" title="How to contact Wikipedia"><span>Contact us</span></a></li>,
 <li class="mw-list-item" id

In [33]:
# create a list with all the texts in the web page of the events
event_description = []
for e in events_txt:
   event_description.append(e.text)
event_description

['Main page',
 'Contents',
 'Current events',
 'Random article',
 'About Wikipedia',
 'Contact us',
 'Donate',
 'Help',
 'Learn to edit',
 'Community portal',
 'Recent changes',
 'Upload file',
 'Create account\n',
 'Log in\n',
 ' Create account',
 ' Log in',
 'Contributions',
 'Talk',
 '\n\n(Top)\n\n',
 '\n\n\n1Events\n\n\n\nToggle Events subsection\n\n\n\n\n\n1.1January\n\n\n\n\n\n\n\n1.2February\n\n\n\n\n\n\n\n1.3March\n\n\n\n\n\n\n\n1.4April\n\n\n\n\n\n\n\n1.5May\n\n\n\n\n\n\n\n1.6June\n\n\n\n\n\n\n\n1.7July\n\n\n\n\n\n\n\n1.8August\n\n\n\n\n\n\n\n1.9September\n\n\n\n\n\n\n\n1.10October\n\n\n\n\n\n\n\n1.11November\n\n\n\n\n\n\n\n1.12December\n\n\n\n\n\n',
 '\n\n\n1.1January\n\n\n\n',
 '\n\n\n1.2February\n\n\n\n',
 '\n\n\n1.3March\n\n\n\n',
 '\n\n\n1.4April\n\n\n\n',
 '\n\n\n1.5May\n\n\n\n',
 '\n\n\n1.6June\n\n\n\n',
 '\n\n\n1.7July\n\n\n\n',
 '\n\n\n1.8August\n\n\n\n',
 '\n\n\n1.9September\n\n\n\n',
 '\n\n\n1.10October\n\n\n\n',
 '\n\n\n1.11November\n\n\n\n',
 '\n\n\n1.12December\n

In [34]:
# create a list of all the specific and caracterizing words contained in the texts events 
event_month = [' January', ' February', ' March', ' April', ' May', ' June',
         ' July', ' August', ' September', ' October', ' November', ' December']

In [35]:
# extract only the meaningful texts from 'li' html tags 
event_date = []

for t in event_description:
    for m in event_month:
        if m in t[:12]:
            event_date.append(t)
event_date    

["1 January\nCroatia adopts the euro and joins the Schengen Area, becoming the 20th member state of the Eurozone and the 27th member of the Schengen Area. This is the first enlargement of the Eurozone since Lithuania's entry in 2015, and the first enlargement of the Schengen Area since Liechtenstein's entry in 2011.[2][3]",
 "3 January – Starting from this date, many countries impose travel restrictions on travel from China due to the relaxation of the country's zero-COVID policy.[4]",
 "5 January – The funeral of Pope Benedict XVI is held at Saint Peter's Square within the Vatican City.[5]",
 '8 January\nThe 2023 Beninese parliamentary election is held, with the Progressive Union for Renewal party winning a plurality of seats.[6]\nCOVID-19 pandemic: China reopens its borders to international visitors, marking the end of travel restrictions that began in March 2020.[7][8]\nFollowing the 2022 Brazilian general election in October that year and the inauguration of Lula da Silva as Presid

In [36]:
# create a function to filter the list for each month and join each element of the list related to the same month
def month_filter(month, arr):
    event_list = list(filter(lambda i : i if month in i[:12] else 0 , event_date))
    sep = '  '
    st = sep.join(event_list)
    return arr.append(st)


In [37]:
# apply the function created in the previous point by passing parameter the distinguishing string and the new list to append the strings divided by month
events_ttl = []
month_filter(' January', events_ttl)
month_filter(' February', events_ttl)
month_filter(' March', events_ttl)
month_filter(' April', events_ttl)
month_filter(' May', events_ttl)
month_filter(' June', events_ttl)
month_filter(' July', events_ttl)
month_filter(' August', events_ttl)
month_filter(' September', events_ttl)
month_filter(' October', events_ttl)
month_filter(' November', events_ttl)
month_filter(' December', events_ttl)


In [38]:
events_ttl

["1 January\nCroatia adopts the euro and joins the Schengen Area, becoming the 20th member state of the Eurozone and the 27th member of the Schengen Area. This is the first enlargement of the Eurozone since Lithuania's entry in 2015, and the first enlargement of the Schengen Area since Liechtenstein's entry in 2011.[2][3]  3 January – Starting from this date, many countries impose travel restrictions on travel from China due to the relaxation of the country's zero-COVID policy.[4]  5 January – The funeral of Pope Benedict XVI is held at Saint Peter's Square within the Vatican City.[5]  8 January\nThe 2023 Beninese parliamentary election is held, with the Progressive Union for Renewal party winning a plurality of seats.[6]\nCOVID-19 pandemic: China reopens its borders to international visitors, marking the end of travel restrictions that began in March 2020.[7][8]\nFollowing the 2022 Brazilian general election in October that year and the inauguration of Lula da Silva as President of Br

In [39]:
# create a new dataframe containing the events of the year divided by month
df_events = pd.DataFrame({'Events': events_ttl})

In [40]:
df_events

,Events
0,1 January\nCroatia adopts the euro and joins t...
1,1 February – Lebanese liquidity crisis: The ce...
2,2 March – The National Assembly of Vietnam dec...
3,2 April\nThe 2023 Bulgarian parliamentary elec...
4,1 May – 2023 banking crisis: San Francisco-bas...
5,"2 June – A train collision in Odisha, India re..."
6,3 July\nIndian oil refiners start payments for...
7,1 August – Global warming: The world's oceans ...
8,1 September – 2023 Singaporean presidential el...
9,"3 October – Elected on 8 January,[221] Kevin M..."


In [41]:
# add the month to the dataframe created to merge the event description into the main dataframe
df_events_month = []

for i in df_events['Events']:
    if ' January' in i[:12]:
        m = 'January'
    elif ' February' in i[:12]:
        m = 'February' 
    elif ' March' in i[:12]:
        m = 'March' 
    elif ' April' in i[:12]:
        m = 'April'
    elif ' May' in i[:12]:
        m = 'May'
    elif ' June' in i[:12]:
        m = 'June' 
    elif ' July' in i[:12]:
        m = 'July' 
    elif ' August' in i[:12]:
        m = 'August' 
    elif ' September' in i[:12]:
        m = 'September' 
    elif ' October' in i[:12]:
        m = 'October' 
    elif ' November' in i[:12]:
        m = 'November' 
    elif ' December' in i[:12]:
        m = 'December'
    else:
        m = 'N/A'    
    df_events_month.append(m)   

df_events['Event month'] = df_events_month

In [42]:
# replace/remove special characters in events descriptions
df_events['Events'].replace('\[(\d+)\]',' ', regex=True, inplace=True)
df_events['Events'] = [i.replace('\n', ' ') for i in df_events['Events']]
df_events

,Events,Event month
0,1 January Croatia adopts the euro and joins th...,January
1,1 February – Lebanese liquidity crisis: The ce...,February
2,2 March – The National Assembly of Vietnam dec...,March
3,2 April The 2023 Bulgarian parliamentary elect...,April
4,1 May – 2023 banking crisis: San Francisco-bas...,May
5,"2 June – A train collision in Odisha, India re...",June
6,3 July Indian oil refiners start payments for ...,July
7,1 August – Global warming: The world's oceans ...,August
8,1 September – 2023 Singaporean presidential el...,September
9,"3 October – Elected on 8 January, Kevin McCar...",October


In [43]:
df_events.to_csv(r'data_update/events.csv', index=False)

# Adding categories from original Survey infos

In [44]:
# add the relation to the 'Moods' on the basis of the Survey relations
mood_relationship = []

for i in survey['Question']:
    if i == 'Please find below some words indicating different sensations and emotions. For each word chose how much you feel this way today' or\
        i == 'Thinking about the future, in a year you\'ll see yourself more or less...' or\
        i == 'How do you see your life in the next 12 months? How likely will the next 12 months be to...':
        s = 'PANAS and Personal Life'
        mood_relationship.append(s)
    elif i ==  "What are the three main moods that characterize the relationships at your home?" or\
        i == "What are the three main moods that characterize the relationships you experience with the people who live near you (e.g., in your neighborhood/town)?" or\
        i == "When you think about Italian current reality (politics, economics, news, culture, etc.) what are the three main states of mind you feel while reading the events?" or\
        i == "When you think about world events (politics, economics, projects, events, etc.) what are the three main states of mind you feel while reading the events?":
        s = 'External Social Context'
        mood_relationship.append(s)
    else:
        s = 'General'
        mood_relationship.append(s)

survey['Mood relationship'] = mood_relationship

In [45]:
# check unique values for 'Topic' column
survey['Topic'].unique()

array(['Active/ Dynamic', 'Upset/ Agitated', 'Hostile/ Contentious',
       'Inspired/ Motivated', 'Uncomfortable/ Embarassed',
       'Smart/ Clear headed', 'Nervoso', 'Determined/ Resolute',
       'Attentive/ Alert', 'Frightened/ Scared', 'Buy a new car',
       'Buy a new house', 'Refurbish your house',
       'Go abroad on jurney (more than a week)', 'Earn more money',
       'Have a new love story', 'Spend a part of your savings',
       'Increase your savings', 'Inherit something', 'Joy', 'Trust',
       'Hope', 'Protection/ Safety', 'Armony', 'Calm', 'Sadness',
       'Indifference', 'Incomprehension', 'Boredom', 'Anger',
       'Anxiety/ Agitation', nan], dtype=object)

In [46]:
# assign a type of 'Moods' in relation to the details of the survey
mood = []

for i in survey['Topic']:
    if i == 'Active/ Dynamic' or i == 'Inspired/ Motivated' or i == 'Smart/ Clear headed' or i == 'Determined/ Resolute' or i == 'Attentive/ Alert'\
        or i ==  'Joy' or i == 'Trust' or i == 'Hope' or i == 'Protection/ Safety' or i == 'Armony' or i == 'Calm':
        s = 'Good'
        mood.append(s)
    elif i == 'Upset/ Agitated' or i == 'Hostile/ Contentious' or i == 'Uncomfortable/ Embarassed' or i == 'Frightened/ Scared' or i == 'Sadness' \
        or i == 'Indifference' or i == 'Incomprehension' or i == 'Boredom' or i == 'Anger' or i == 'Anxiety/ Agitation':
        s = 'Bad'
        mood.append(s)
    elif i == 'Buy a new car' or i == 'Buy a new house' or i == 'Refurbish your house' or i == "Go abroad on jurney (more than a week)"\
         or i == 'Earn more money' or i == 'Have a new love story' or i == 'Spend a part of your savings' \
         or i == 'Increase your savings' or i == "Inherit something":
        s = 'Future'
        mood.append(s)
    else:
        s = 'Global'
        mood.append(s)
        
survey["General Mood"] = mood

In [47]:
survey.head()

,User ID,Variable,Question,Topic,Answer Label,Answer Value,Age,CAP,City,Num. of Citizens,...,Num. of Children (0-14 years),Season,Month,Time Slot,birth_year,Survey filling date,Survey filling hour,Weekday,Mood relationship,General Mood
0,221250719,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A lot,5,20,70032,BA,56929.0,...,0,Winter,February,Evening,2003,14/02/2023,19:06:44,Tuesday,PANAS and Personal Life,Good
1,89508748,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little yes and a little no,3,21,87060,CS,1281.0,...,0,Winter,February,Evening,2002,27/02/2023,18:36:02,Monday,PANAS and Personal Life,Good
2,98143773,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little,2,21,63822,FM,16062.0,...,0,Winter,February,Evening,2002,19/02/2023,18:39:59,Sunday,PANAS and Personal Life,Good
3,641447689,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little,2,21,90024,PA,7614.0,...,1,Winter,February,Evening,2002,28/02/2023,20:32:30,Tuesday,PANAS and Personal Life,Good
4,61843325,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little,2,21,95024,CT,50190.0,...,0,Winter,February,Evening,2002,16/02/2023,18:25:16,Thursday,PANAS and Personal Life,Good


# Adding Geographical variables

https://www.istat.it/it/archivio/156224 download  Classificazioni statistiche – anni 2022-2023 (zip)
https://www.gardainformatica.it/database-comuni-italiani#scarica

In [48]:
# read first csv with geographical details, downloaded from ISTAT website
lat_long = pd.read_csv(r'sources/gi_comuni_cap.csv')
print(lat_long.columns)
lat_long.head()

Index(['codice_istat', 'denominazione_ita_altra', 'denominazione_ita',
       'denominazione_altra', 'cap', 'sigla_provincia',
       'denominazione_provincia', 'tipologia_provincia', 'codice_regione',
       'denominazione_regione', 'tipologia_regione', 'ripartizione_geografica',
       'flag_capoluogo', 'codice_belfiore', 'lat', 'lon', 'superficie_kmq',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'],
      dtype='object')


,codice_istat,denominazione_ita_altra,denominazione_ita,denominazione_altra,cap,sigla_provincia,denominazione_provincia,tipologia_provincia,codice_regione,denominazione_regione,tipologia_regione,ripartizione_geografica,flag_capoluogo,codice_belfiore,lat,lon,superficie_kmq,Unnamed: 17,Unnamed: 18,Unnamed: 19
0,1001,Agliè,Agliè,NaN,10011,TO,Torino,Citta metropolitana,1,Piemonte,statuto ordinario,Nord-ovest,NO,A074,45,3634368,7,7685999,13,1462
1,1002,Airasca,Airasca,NaN,10060,TO,Torino,Citta metropolitana,1,Piemonte,statuto ordinario,Nord-ovest,NO,A109,44,9168832,7,4844311,15,7393
2,1003,Ala di Stura,Ala di Stura,NaN,10070,TO,Torino,Citta metropolitana,1,Piemonte,statuto ordinario,Nord-ovest,NO,A117,45,3151133,7,3043438,46,3315
3,1004,Albiano d'Ivrea,Albiano d'Ivrea,NaN,10010,TO,Torino,Citta metropolitana,1,Piemonte,statuto ordinario,Nord-ovest,NO,A157,45,4338982,7,9491448,11,7314
4,1006,Almese,Almese,NaN,10040,TO,Torino,Citta metropolitana,1,Piemonte,statuto ordinario,Nord-ovest,NO,A218,45,1176634,7,3951819,17,8756


In [49]:
# keep only relevant columns
lat_long = lat_long[['codice_istat', 'denominazione_ita', 'cap', 'sigla_provincia', 'denominazione_provincia', 'denominazione_regione', 'tipologia_regione', 'lat', 'lon', 'superficie_kmq', 'Unnamed: 17']]
lat_long = lat_long.copy()
lat_long.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8463 entries, 0 to 8462
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   codice_istat             8463 non-null   int64 
 1   denominazione_ita        8463 non-null   object
 2   cap                      8463 non-null   int64 
 3   sigla_provincia          8347 non-null   object
 4   denominazione_provincia  8463 non-null   object
 5   denominazione_regione    8463 non-null   object
 6   tipologia_regione        8463 non-null   object
 7   lat                      8463 non-null   int64 
 8   lon                      8463 non-null   int64 
 9   superficie_kmq           8463 non-null   int64 
 10  Unnamed: 17              8463 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 727.4+ KB


In [50]:
# generate latitude column
latitude = []

for a, b in zip(lat_long['lat'], lat_long['lon']):
        l = str(a) + '.' + str(b)
        latitude.append(l)

In [51]:
# generate longitude column
longitude = []

for a, b in zip(lat_long['superficie_kmq'], lat_long['Unnamed: 17']):
        l = str(a) + '.' + str(b)
        longitude.append(l)

In [52]:
lat_long['Latitude'] = latitude
lat_long['Longitude'] = longitude
lat_long.drop(columns=['lat', 'lon', 'superficie_kmq', 'Unnamed: 17'], inplace=True)
lat_long

,codice_istat,denominazione_ita,cap,sigla_provincia,denominazione_provincia,denominazione_regione,tipologia_regione,Latitude,Longitude
0,1001,Agliè,10011,TO,Torino,Piemonte,statuto ordinario,45.3634368,7.7685999
1,1002,Airasca,10060,TO,Torino,Piemonte,statuto ordinario,44.9168832,7.4844311
2,1003,Ala di Stura,10070,TO,Torino,Piemonte,statuto ordinario,45.3151133,7.3043438
3,1004,Albiano d'Ivrea,10010,TO,Torino,Piemonte,statuto ordinario,45.4338982,7.9491448
4,1006,Almese,10040,TO,Torino,Piemonte,statuto ordinario,45.1176634,7.3951819
...,...,...,...,...,...,...,...,...,...
8458,111103,Villaputzu,9040,SU,Sud Sardegna,Sardegna,statuto speciale,39.5346598,9.5514436
8459,111104,Villasalto,9040,SU,Sud Sardegna,Sardegna,statuto speciale,39.4680466,9.3941121
8460,111105,Villasimius,9049,SU,Sud Sardegna,Sardegna,statuto speciale,39.1550781,9.5099547
8461,111106,Villasor,9034,SU,Sud Sardegna,Sardegna,statuto speciale,39.3833349,8.9108004


In [53]:
# read second csv containing geographical details downloaded from ISTAT
location_info = pd.read_excel(r'sources/Classificazioni statistiche-e-dimensione-dei-comuni_30_06_2023.xlsx')
location_info.head()

,Codice Regione,Codice Istat del Comune \n(alfanumerico),Codice Istat del Comune \n(numerico),Denominazione (Italiana e straniera),Denominazione altra lingua,Superficie territoriale (kmq) al 01/01/2023,Popolazione legale 2021 (31/12/2021),Popolazione residente al 31/12/2022,Zona altimetrica,Altitudine del centro (metri),Comune litoraneo,Comune isolano,Zone costiere,Grado di urbanizzazione
0,1,1001,1001,Agliè,NaN,13.1462,2562,2558,3,315,0,0,0,2
1,1,1002,1002,Airasca,NaN,15.7393,3660,3681,5,257,0,0,0,3
2,1,1003,1003,Ala di Stura,NaN,46.3315,467,465,1,1080,0,0,0,3
3,1,1004,1004,Albiano d'Ivrea,NaN,11.7314,1637,1628,3,230,0,0,0,3
4,1,1006,1006,Almese,NaN,17.8756,6331,6280,3,364,0,0,0,2


In [54]:
# keep only relevant columns
location_info = location_info[['Codice Istat del Comune \n(alfanumerico)', 'Zona altimetrica', 'Comune litoraneo','Grado di urbanizzazione']]
location_info = location_info.copy()
location_info.head(3)


,Codice Istat del Comune \n(alfanumerico),Zona altimetrica,Comune litoraneo,Grado di urbanizzazione
0,1001,3,0,2
1,1002,5,0,3
2,1003,1,0,3


In [55]:
# assign 'location altitude' from ISTAT tables
altitude_profile = []

for i in location_info['Zona altimetrica']:
    if i == 1:
        z = 'Inner Mountain'
    elif i == 2:
        z = 'Coastal Mountain'
    elif i == 3:
        z = 'Inner Hill'
    elif i == 4:
        z = 'Coastal Hill'
    elif i == 5:
        z = 'Plain'
    altitude_profile.append(z)

location_info['Altitude Profile'] = altitude_profile

In [56]:
# assign the type of position of each city
location_position = []

for l in location_info['Comune litoraneo']:
    if l == 1:
        m = 'Coastline'
    else:
        m = 'Hinterland'
    location_position.append(m)

location_info['Geo Position'] = location_position

In [57]:
# assign the urbanzation rate
urbanization_level = []

for g in location_info['Grado di urbanizzazione']:
    if g == 1:
        z = 'City / Highly populated'
    elif g == 2:
        z = 'Small city or suburb / Intermediately populated' 
    elif g == 3:
        z = 'Rural / Slightly populated'
    urbanization_level.append(z)

location_info['Urbanization Level'] = urbanization_level

In [58]:
location_info.columns

Index(['Codice Istat del Comune \n(alfanumerico)', 'Zona altimetrica',
       'Comune litoraneo', 'Grado di urbanizzazione', 'Altitude Profile',
       'Geo Position', 'Urbanization Level'],
      dtype='object')

In [59]:
# keep only relevant columns
location_info = location_info[['Codice Istat del Comune \n(alfanumerico)', 'Altitude Profile', 'Geo Position', 'Urbanization Level']]
location_info.head()

,Codice Istat del Comune \n(alfanumerico),Altitude Profile,Geo Position,Urbanization Level
0,1001,Inner Hill,Hinterland,Small city or suburb / Intermediately populated
1,1002,Plain,Hinterland,Rural / Slightly populated
2,1003,Inner Mountain,Hinterland,Rural / Slightly populated
3,1004,Inner Hill,Hinterland,Rural / Slightly populated
4,1006,Inner Hill,Hinterland,Small city or suburb / Intermediately populated


In [60]:
lat_long.head()

,codice_istat,denominazione_ita,cap,sigla_provincia,denominazione_provincia,denominazione_regione,tipologia_regione,Latitude,Longitude
0,1001,Agliè,10011,TO,Torino,Piemonte,statuto ordinario,45.3634368,7.7685999
1,1002,Airasca,10060,TO,Torino,Piemonte,statuto ordinario,44.9168832,7.4844311
2,1003,Ala di Stura,10070,TO,Torino,Piemonte,statuto ordinario,45.3151133,7.3043438
3,1004,Albiano d'Ivrea,10010,TO,Torino,Piemonte,statuto ordinario,45.4338982,7.9491448
4,1006,Almese,10040,TO,Torino,Piemonte,statuto ordinario,45.1176634,7.3951819


In [61]:
# merge the two tables containing the geographical details together for more information
geo_details = pd.merge(lat_long, location_info, how='left', left_on='codice_istat', right_on='Codice Istat del Comune \n(alfanumerico)')

In [62]:
# drop irrelevant columns
geo_details.drop(columns=['codice_istat', 'Codice Istat del Comune \n(alfanumerico)'], inplace=True)
geo_details['cap'] = [str(i) for i in geo_details['cap']]
print(geo_details.info())
geo_details.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8463 entries, 0 to 8462
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   denominazione_ita        8463 non-null   object
 1   cap                      8463 non-null   object
 2   sigla_provincia          8347 non-null   object
 3   denominazione_provincia  8463 non-null   object
 4   denominazione_regione    8463 non-null   object
 5   tipologia_regione        8463 non-null   object
 6   Latitude                 8463 non-null   object
 7   Longitude                8463 non-null   object
 8   Altitude Profile         8463 non-null   object
 9   Geo Position             8463 non-null   object
 10  Urbanization Level       8463 non-null   object
dtypes: object(11)
memory usage: 793.4+ KB
None


,denominazione_ita,cap,sigla_provincia,denominazione_provincia,denominazione_regione,tipologia_regione,Latitude,Longitude,Altitude Profile,Geo Position,Urbanization Level
0,Agliè,10011,TO,Torino,Piemonte,statuto ordinario,45.3634368,7.7685999,Inner Hill,Hinterland,Small city or suburb / Intermediately populated
1,Airasca,10060,TO,Torino,Piemonte,statuto ordinario,44.9168832,7.4844311,Plain,Hinterland,Rural / Slightly populated
2,Ala di Stura,10070,TO,Torino,Piemonte,statuto ordinario,45.3151133,7.3043438,Inner Mountain,Hinterland,Rural / Slightly populated
3,Albiano d'Ivrea,10010,TO,Torino,Piemonte,statuto ordinario,45.4338982,7.9491448,Inner Hill,Hinterland,Rural / Slightly populated
4,Almese,10040,TO,Torino,Piemonte,statuto ordinario,45.1176634,7.3951819,Inner Hill,Hinterland,Small city or suburb / Intermediately populated


In [63]:
survey.head(2)

,User ID,Variable,Question,Topic,Answer Label,Answer Value,Age,CAP,City,Num. of Citizens,...,Num. of Children (0-14 years),Season,Month,Time Slot,birth_year,Survey filling date,Survey filling hour,Weekday,Mood relationship,General Mood
0,221250719,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A lot,5,20,70032,BA,56929.0,...,0,Winter,February,Evening,2003,14/02/2023,19:06:44,Tuesday,PANAS and Personal Life,Good
1,89508748,B11_Q62_A_1,Please find below some words indicating differ...,Active/ Dynamic,A little yes and a little no,3,21,87060,CS,1281.0,...,0,Winter,February,Evening,2002,27/02/2023,18:36:02,Monday,PANAS and Personal Life,Good


In [64]:
# rename columns
geo_details.rename(columns={'denominazione_ita':'Municipality', 'denominazione_provincia':'City Name', 'denominazione_regione':'Region',
                       'tipologia_regione':'Region Type', 'cap':'City CAP' }, inplace=True)

In [65]:
geo_details['Region Type'].replace({'statuto ordinario':'Ordinary Statute', 'statuto speciale':'Special Statute'}, inplace= True)

In [66]:
# save the dataframe to csv into the directory to update future surveys
geo_details.to_csv(r'data_update/geographical_details.csv', index=False)

# Save Final Dataframe to CSV

In [67]:
# save final dataframe to csv
survey.to_csv(r'data_clean/03_survey_to_merge.csv', index=False)